<a href="https://colab.research.google.com/github/Lishat/Transfer-Learning-on-CIFAR-10-dataset/blob/master/Transfer_Learning_using_Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.optimizers import Adam

In [2]:
from tensorflow.keras.datasets import cifar10

(x_train, y_train), (x_test, y_test) = cifar10.load_data()

170500096/170498071 [==============================] - 2s 0us/step


In [0]:
from tensorflow.keras.applications.resnet50 import ResNet50

In [4]:
pre_trained_model = ResNet50(input_shape=(32, 32, 3), include_top=False, weights='imagenet')

/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


94658560/94653016 [==============================] - 1s 0us/step


In [5]:
pre_trained_model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 38, 38, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 16, 16, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 16, 16, 64)   256         conv1[0][0]                      
___________________________________________________________________________________________

In [0]:
for layer in pre_trained_model.layers:
  layer.trainable = False

In [7]:
pre_trained_model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 38, 38, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 16, 16, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 16, 16, 64)   256         conv1[0][0]                      
___________________________________________________________________________________________

In [8]:

last_layer = pre_trained_model.get_layer('activation_48')
print(last_layer.output_shape)

(None, 1, 1, 2048)


In [9]:
x = layers.Flatten()(last_layer.output)
x = layers.Dense(1024, activation = 'relu')(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(512, activation = 'relu')(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(256, activation = 'relu')(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(128, activation = 'relu')(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(64, activation = 'relu')(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(32, activation = 'relu')(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(10, activation = 'softmax')(x)
final_model = Model(pre_trained_model.input, x)

W0702 06:06:13.765776 140717400463232 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [10]:
final_model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 38, 38, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 16, 16, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 16, 16, 64)   256         conv1[0][0]                      
______________________________________________________________________________________________

In [0]:
final_model.compile(optimizer = Adam(lr=1e-4), 
              loss = 'categorical_crossentropy', 
              metrics = ['acc'])

In [0]:
import numpy as np
new_y_train = np.zeros((y_train.shape[0], 10))
for i in range(len(y_train)):
  new_y_train[i][y_train[i]] = 1
y_train = new_y_train
new_y_test = np.zeros((y_test.shape[0], 10))
for i in range(len(y_test)):
  new_y_test[i][y_test[i]] = 1
y_test = new_y_test

In [14]:
history = final_model.fit(x=x_train, y=y_train, epochs=50, batch_size=64, validation_data=(x_test, y_test))

Train on 50000 samples, validate on 10000 samples
Epoch 1/50
50000/50000 [==============================] - 21s 427us/sample - loss: 2.1631 - acc: 0.1942 - val_loss: 2.6133 - val_acc: 0.1930
Epoch 2/50
50000/50000 [==============================] - 15s 306us/sample - loss: 1.8630 - acc: 0.3272 - val_loss: 3.4669 - val_acc: 0.1810
Epoch 3/50
50000/50000 [==============================] - 15s 307us/sample - loss: 1.7312 - acc: 0.3903 - val_loss: 2.7563 - val_acc: 0.2497
Epoch 4/50
50000/50000 [==============================] - 15s 310us/sample - loss: 1.6583 - acc: 0.4250 - val_loss: 3.0552 - val_acc: 0.2326
Epoch 5/50
50000/50000 [==============================] - 15s 308us/sample - loss: 1.5939 - acc: 0.4508 - val_loss: 3.0245 - val_acc: 0.2408
Epoch 6/50
50000/50000 [==============================] - 16s 316us/sample - loss: 1.5394 - acc: 0.4725 - val_loss: 3.1475 - val_acc: 0.2362
Epoch 7/50
50000/50000 [==============================] - 16s 310us/sample - loss: 1.5048 - acc: 0.4905 